# 📊 하수도 인프라 지수 분석

## 📋 분석 개요

이 노트북은 한국환경공단의 하수도 보급률 데이터를 분석하여 지역별 하수도 인프라 지수를 계산하고 시각화합니다.

### 🎯 분석 목표
- 지역별 하수도 인프라 현황 파악
- 하수도 인프라 지수 개발 및 등급 분류
- 지역간 인프라 격차 분석
- 시각화를 통한 인사이트 도출

### 📊 주요 지표
- 하수도 설치율
- 공공하수처리구역 인구보급률
- 고도처리인구 보급률
- 인구 밀도

### 🏆 인프라 지수 구성
- 하수도설치율 (30%)
- 공공하수처리구역 인구보급률 (30%)
- 고도처리인구 보급률 (20%)
- 인구밀도 정규화 (20%)

In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import folium
import geopandas as gpd
import json
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
import matplotlib.font_manager as fm

# 나눔고딕 폰트 경로 설정
project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
font_path = os.path.join(project_dir, 'data', 'NanumGothic.ttf')

# 폰트 등록 및 설정 (파일이 존재하는 경우에만)
if os.path.exists(font_path):
    fm.fontManager.addfont(font_path)
    plt.rcParams['font.family'] = 'NanumGothic'
    print(f"폰트 설정 완료: {font_path}")
else:
    print(f"폰트 파일을 찾을 수 없습니다: {font_path}")
    plt.rcParams['font.family'] = 'DejaVu Sans'  # 기본 폰트 사용
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 임포트 완료")

## 📁 전처리된 데이터 로드

In [ ]:
# 전처리된 데이터 로드
import os
# 현재 작업 디렉토리 확인
print(f"현재 작업 디렉토리: {os.getcwd()}")
# 절대 경로 사용
project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
file_path = os.path.join(project_dir, "data", "processed", "sewer_infrastructure_processed.csv")
print(f"절대 경로: {file_path}")
print(f"파일 존재 여부: {os.path.exists(file_path)}")
df = pd.read_csv(file_path, encoding='utf-8')

print(f"전처리된 데이터 로드 완료: {len(df)}개 행, {len(df.columns)}개 컬럼")
print(f"컬럼명: {list(df.columns)}")
print(f"\n시도별 데이터 개수:")
print(df['시도'].value_counts())
df.head()

## 🔍 데이터 탐색

In [ ]:
# 기본 정보 확인
print("=== 데이터 기본 정보 ===")
print(df.info())

print("\n=== 기술 통계 ===")
print(df.describe())

print("\n=== 결측값 현황 ===")
print(df.isnull().sum())

## 📊 하수도 인프라 지수 계산

In [ ]:
# 가중치 설정
weights = {
    '하수도설치율': 0.3,
    '공공하수처리구역 인구보급률': 0.3,
    '고도처리인구 보급률': 0.2,
    '인구밀도_정규화': 0.2
}

# 인구 밀도 정규화 (0-100 스케일)
scaler = MinMaxScaler(feature_range=(0, 100))
df['인구밀도_정규화'] = scaler.fit_transform(df[['인구밀도']])

# 가중 평균으로 인프라 지수 계산
df['하수도_인프라_지수'] = (
    df['하수도설치율'] * weights['하수도설치율'] +
    df['공공하수처리구역 인구보급률'] * weights['공공하수처리구역 인구보급률'] +
    df['고도처리인구 보급률'] * weights['고도처리인구 보급률'] +
    df['인구밀도_정규화'] * weights['인구밀도_정규화']
)

# 지수 등급 분류
df['인프라_등급'] = pd.cut(
    df['하수도_인프라_지수'],
    bins=[0, 40, 60, 80, 100],
    labels=['매우 낮음', '낮음', '보통', '높음'],
    include_lowest=True
)

print("하수도 인프라 지수 계산 완료")
print(f"평균 인프라 지수: {df['하수도_인프라_지수'].mean():.2f}")
print(f"최고 인프라 지수: {df['하수도_인프라_지수'].max():.2f}")
print(f"최저 인프라 지수: {df['하수도_인프라_지수'].min():.2f}")

df.head()

## 📈 시각화

In [ ]:
# 1. 인프라 지수 분포
plt.figure(figsize=(15, 10))

plt.subplot(2, 3, 1)
plt.hist(df['하수도_인프라_지수'], bins=30, alpha=0.7, color='skyblue')
plt.title('하수도 인프라 지수 분포')
plt.xlabel('인프라 지수')
plt.ylabel('빈도')

# 2. 등급별 분포
plt.subplot(2, 3, 2)
grade_counts = df['인프라_등급'].value_counts()
plt.pie(grade_counts.values, labels=grade_counts.index, autopct='%1.1f%%')
plt.title('인프라 등급별 분포')

# 3. 시도별 평균 지수
plt.subplot(2, 3, 3)
region_means = df.groupby('시도')['하수도_인프라_지수'].mean().sort_values(ascending=True)
plt.barh(range(len(region_means)), region_means.values)
plt.yticks(range(len(region_means)), region_means.index)
plt.title('시도별 평균 하수도 인프라 지수')
plt.xlabel('평균 인프라 지수')

# 4. 지표별 상관관계
plt.subplot(2, 3, 4)
correlation_cols = ['하수도설치율', '공공하수처리구역 인구보급률', '고도처리인구 보급률', '하수도_인프라_지수']
correlation_matrix = df[correlation_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('지표별 상관관계')

# 5. 인구 밀도 vs 인프라 지수
plt.subplot(2, 3, 5)
plt.scatter(df['인구밀도'], df['하수도_인프라_지수'], alpha=0.6)
plt.xlabel('인구 밀도 (명/km²)')
plt.ylabel('하수도 인프라 지수')
plt.title('인구 밀도 vs 인프라 지수')

# 6. 등급별 인구 밀도 분포
plt.subplot(2, 3, 6)
sns.boxplot(data=df, x='인프라_등급', y='인구밀도')
plt.title('등급별 인구 밀도 분포')
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

## 🏆 상위/하위 지역 분석

In [ ]:
# 상위 20개 지역
top_regions = df.nlargest(20, '하수도_인프라_지수')[
    ['시도', '행정구역명', '하수도_인프라_지수', '인프라_등급', '하수도설치율', '공공하수처리구역 인구보급률']
]

print("=== 상위 20개 지역 ===")
print(top_regions)

# 하위 20개 지역
bottom_regions = df.nsmallest(20, '하수도_인프라_지수')[
    ['시도', '행정구역명', '하수도_인프라_지수', '인프라_등급', '하수도설치율', '공공하수처리구역 인구보급률']
]

print("\n=== 하위 20개 지역 ===")
print(bottom_regions)

## 🏛️ 시도별 분석

In [ ]:
# 시도별 통계
region_stats = df.groupby('시도').agg({
    '하수도_인프라_지수': ['mean', 'std', 'min', 'max', 'count'],
    '하수도설치율': 'mean',
    '공공하수처리구역 인구보급률': 'mean',
    '고도처리인구 보급률': 'mean',
    '인구밀도': 'mean'
}).round(2)

region_stats.columns = [
    '평균_인프라지수', '표준편차', '최소값', '최대값', '지역수',
    '평균_하수도설치율', '평균_공공하수처리구역', '평균_고도처리', '평균_인구밀도'
]

print("=== 시도별 하수도 인프라 통계 ===")
print(region_stats.sort_values('평균_인프라지수', ascending=False))

# 시도별 등급 분포
grade_by_region = pd.crosstab(df['시도'], df['인프라_등급'])
print("\n=== 시도별 인프라 등급 분포 ===")
print(grade_by_region)

## 🗺️ 인터랙티브 지도 시각화

In [ ]:
# 시도별 평균 하수도 인프라 지수 계산
region_infrastructure = df.groupby('시도')['하수도_인프라_지수'].mean().reset_index()
region_infrastructure.columns = ['시도', '평균_인프라_지수']

# 시도명 매핑 (지도 데이터와 맞추기 위해)
region_mapping = {
    '서울특별시': '서울특별시',
    '부산광역시': '부산광역시',
    '대구광역시': '대구광역시',
    '인천광역시': '인천광역시',
    '광주광역시': '광주광역시',
    '대전광역시': '대전광역시',
    '울산광역시': '울산광역시',
    '세종특별자치시': '세종특별자치시',
    '경기도': '경기도',
    '강원특별자치도': '강원도',
    '충청북도': '충청북도',
    '충청남도': '충청남도',
    '전라북도': '전라북도',
    '전라남도': '전라남도',
    '경상북도': '경상북도',
    '경상남도': '경상남도',
    '제주특별자치도': '제주특별자치도'
}

region_infrastructure['지도_시도명'] = region_infrastructure['시도'].map(region_mapping)

print("=== 시도별 평균 하수도 인프라 지수 ===")
print(region_infrastructure.sort_values('평균_인프라_지수', ascending=False))

# 대한민국 지도 GeoJSON 데이터 (간단한 버전)
korea_geojson = {
    "type": "FeatureCollection",
    "features": [
        {"type": "Feature", "properties": {"name": "서울특별시"}, "geometry": {"type": "Polygon", "coordinates": [[[126.8, 37.4], [127.2, 37.4], [127.2, 37.7], [126.8, 37.7], [126.8, 37.4]]]}},
        {"type": "Feature", "properties": {"name": "부산광역시"}, "geometry": {"type": "Polygon", "coordinates": [[[128.9, 35.0], [129.3, 35.0], [129.3, 35.3], [128.9, 35.3], [128.9, 35.0]]]}},
        {"type": "Feature", "properties": {"name": "대구광역시"}, "geometry": {"type": "Polygon", "coordinates": [[[128.4, 35.7], [128.8, 35.7], [128.8, 36.0], [128.4, 36.0], [128.4, 35.7]]]}},
        {"type": "Feature", "properties": {"name": "인천광역시"}, "geometry": {"type": "Polygon", "coordinates": [[[126.4, 37.3], [126.8, 37.3], [126.8, 37.6], [126.4, 37.6], [126.4, 37.3]]]}},
        {"type": "Feature", "properties": {"name": "광주광역시"}, "geometry": {"type": "Polygon", "coordinates": [[[126.7, 35.0], [127.1, 35.0], [127.1, 35.3], [126.7, 35.3], [126.7, 35.0]]]}},
        {"type": "Feature", "properties": {"name": "대전광역시"}, "geometry": {"type": "Polygon", "coordinates": [[[127.2, 36.2], [127.6, 36.2], [127.6, 36.5], [127.2, 36.5], [127.2, 36.2]]]}},
        {"type": "Feature", "properties": {"name": "울산광역시"}, "geometry": {"type": "Polygon", "coordinates": [[[129.2, 35.4], [129.6, 35.4], [129.6, 35.7], [129.2, 35.7], [129.2, 35.4]]]}},
        {"type": "Feature", "properties": {"name": "세종특별자치시"}, "geometry": {"type": "Polygon", "coordinates": [[[127.1, 36.4], [127.5, 36.4], [127.5, 36.7], [127.1, 36.7], [127.1, 36.4]]]}},
        {"type": "Feature", "properties": {"name": "경기도"}, "geometry": {"type": "Polygon", "coordinates": [[[126.5, 37.0], [127.5, 37.0], [127.5, 38.0], [126.5, 38.0], [126.5, 37.0]]]}},
        {"type": "Feature", "properties": {"name": "강원도"}, "geometry": {"type": "Polygon", "coordinates": [[[127.5, 37.5], [129.0, 37.5], [129.0, 38.5], [127.5, 38.5], [127.5, 37.5]]]}},
        {"type": "Feature", "properties": {"name": "충청북도"}, "geometry": {"type": "Polygon", "coordinates": [[[127.0, 36.5], [128.5, 36.5], [128.5, 37.5], [127.0, 37.5], [127.0, 36.5]]]}},
        {"type": "Feature", "properties": {"name": "충청남도"}, "geometry": {"type": "Polygon", "coordinates": [[[126.0, 36.0], [127.5, 36.0], [127.5, 37.0], [126.0, 37.0], [126.0, 36.0]]]}},
        {"type": "Feature", "properties": {"name": "전라북도"}, "geometry": {"type": "Polygon", "coordinates": [[[126.5, 35.5], [128.0, 35.5], [128.0, 36.5], [126.5, 36.5], [126.5, 35.5]]]}},
        {"type": "Feature", "properties": {"name": "전라남도"}, "geometry": {"type": "Polygon", "coordinates": [[[126.0, 34.5], [127.5, 34.5], [127.5, 35.5], [126.0, 35.5], [126.0, 34.5]]]}},
        {"type": "Feature", "properties": {"name": "경상북도"}, "geometry": {"type": "Polygon", "coordinates": [[[128.0, 35.5], [130.0, 35.5], [130.0, 37.0], [128.0, 37.0], [128.0, 35.5]]]}},
        {"type": "Feature", "properties": {"name": "경상남도"}, "geometry": {"type": "Polygon", "coordinates": [[[127.5, 34.5], [129.5, 34.5], [129.5, 36.0], [127.5, 36.0], [127.5, 34.5]]]}},
        {"type": "Feature", "properties": {"name": "제주특별자치도"}, "geometry": {"type": "Polygon", "coordinates": [[[126.0, 33.0], [127.0, 33.0], [127.0, 34.0], [126.0, 34.0], [126.0, 33.0]]]}}
    ]
}

# 지도 생성
m = folium.Map(location=[36.5, 127.5], zoom_start=7, tiles='OpenStreetMap')

# 색상 스케일 생성
min_value = region_infrastructure['평균_인프라_지수'].min()
max_value = region_infrastructure['평균_인프라_지수'].max()

def get_color(value):
    """인프라 지수에 따른 색상 반환"""
    if value >= 80:
        return '#1f77b4'  # 파랑 (높음)
    elif value >= 60:
        return '#2ca02c'  # 초록 (보통)
    elif value >= 40:
        return '#ff7f0e'  # 주황 (낮음)
    else:
        return '#d62728'  # 빨강 (매우 낮음)

# Choropleth 지도 생성
folium.Choropleth(
    geo_data=korea_geojson,
    name='choropleth',
    data=region_infrastructure,
    columns=['지도_시도명', '평균_인프라_지수'],
    key_on='feature.properties.name',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='하수도 인프라 지수',
    bins=5
).add_to(m)

# 툴팁 추가
for idx, row in region_infrastructure.iterrows():
    folium.GeoJson(
        korea_geojson,
        name='tooltip',
        style_function=lambda x, region_name=row['지도_시도명']: {
            'fillColor': get_color(row['평균_인프라_지수']) if x['properties']['name'] == region_name else 'transparent',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.7
        },
        tooltip=folium.Tooltip(
            f"<b>{row['시도']}</b><br>"
            f"평균 인프라 지수: {row['평균_인프라_지수']:.2f}<br>"
            f"등급: {"높음" if row['평균_인프라_지수'] >= 80 else "보통" if row['평균_인프라_지수'] >= 60 else "낮음" if row['평균_인프라_지수'] >= 40 else "매우 낮음"}",
            style="background-color: white; border: 2px solid black; border-radius: 3px; padding: 5px;"
        )
    ).add_to(m)

# 레이어 컨트롤 추가
folium.LayerControl().add_to(m)

# 지도 저장
map_file = os.path.join(project_dir, "results", "sewer_infrastructure_map.html")
m.save(map_file)

print(f"인터랙티브 지도 생성 완료: {map_file}")
print("\n지도를 웹 브라우저에서 열어보세요!")

# 지도 표시
m

## 💾 결과 저장

In [ ]:
# 결과 디렉토리 생성
import os
# 절대 경로 사용
project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
results_dir = os.path.join(project_dir, "results")
processed_dir = os.path.join(project_dir, "data", "processed")

os.makedirs(results_dir, exist_ok=True)
os.makedirs(processed_dir, exist_ok=True)

# 분석 결과 저장
analysis_file = os.path.join(processed_dir, "sewer_infrastructure_analysis.csv")
region_file = os.path.join(results_dir, "sewer_infrastructure_by_region.csv")

df.to_csv(analysis_file, index=False, encoding='utf-8-sig')
region_stats.to_csv(region_file, encoding='utf-8-sig')

print("=== 분석 결과 저장 완료 ===")
print(f"1. 전체 분석 결과: {analysis_file}")
print(f"2. 시도별 통계: {region_file}")
print(f"3. 인터랙티브 지도: {map_file}")

# 요약 통계 출력
print(f"\n=== 분석 요약 ===")
print(f"총 분석 지역: {len(df)}개")
print(f"평균 하수도 인프라 지수: {df['하수도_인프라_지수'].mean():.2f}")
